#**Smart Wardrobe: Integrating YOLOv8 and Large Language Models in Fashion Styling.**

###1. Install YOLOv8 and Import YOLO and other Dependencies

In [ ]:
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import numpy as np
import yaml
from torchvision.datasets import FashionMNIST
from torchvision import transforms
import os
from PIL import Image
import requests
from io import BytesIO
import torch
import openai
from openai import OpenAI
import requests
from io import BytesIO
from google.colab import files

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.5/898.5 kB 20.2 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


###2. Model Setup

In [ ]:
#Load the model
model = YOLO("yolov8n-cls.yaml")

YOLOv8n-cls summary: 99 layers, 2,719,288 parameters, 2,719,288 gradients, 4.4 GFLOPs


###3. Training and testing the Model

During training, YOLO internally validates the model. This process ensures that the model isn't just memorizing the training data but is generalizing well to unseen examples. This process will also download the FashionMnist dataset.

In [ ]:
#Train the model
results = model.train(data="fashion-mnist", epochs=10, imgsz=28)

Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=classify, mode=train, model=yolov8n-cls.yaml, data=fashion-mnist, epochs=10, time=None, patience=100, batch=16, imgsz=28, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, li

100%|██████████| 47.0M/47.0M [00:00<00:00, 83.9MB/s]
Unzipping /content/datasets/fashion-mnist.zip to /content/datasets/fashion-mnist...: 100%|██████████| 70023/70023 [00:20<00:00, 3383.94file/s]


Dataset download success ✅ (26.7s), saved to /content/datasets/fashion-mnist

train: /content/datasets/fashion-mnist/train... found 60000 images in 10 classes ✅ 
val: None...
test: /content/datasets/fashion-mnist/test... found 10000 images in 10 classes ✅ 
Overriding model.yaml nc=1000 with nc=10

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5              

train: Scanning /content/datasets/fashion-mnist/train... 60000 images, 0 corrupt: 100%|██████████| 60000/60000 [00:16<00:00, 3739.42it/s]


train: New cache created: /content/datasets/fashion-mnist/train.cache


...

      10/10         0G     0.5201         16         32: 100%|██████████| 3750/3750 [06:18<00:00,  9.92it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 313/313 [00:14<00:00, 21.77it/s]

                   all      0.879      0.999



10 epochs completed in 1.138 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 3.0MB
Optimizer stripped from runs/classify/train/weights/best.pt, 3.0MB

Validating runs/classify/train/weights/best.pt...
WARNING ⚠️ validating an untrained model YAML will result in 0 mAP.
Ultralytics 8.3.40 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLOv8n-cls summary (fused): 73 layers, 1,447,690 parameters, 0 gradients, 3.3 GFLOPs
WARNING ⚠️ Dataset 'split=val' not found, using 'split=test' instead.
train: /content/datasets/fashion-mnist/train... found 60000 images in 10 classes ✅ 
val: None...
test: /content/datasets/fashion-mnist/test... found 10000 images in 10 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 313/313 [00:13<00:00, 23.03it/s]


                   all      0.879      0.999
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train


### Running Inference on the model Using some samples from the test data
Defined the class labels for Fashion MNIST

In [ ]:
# Define the manual class labels for Fashion MNIST
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle Boot"
]

In [ ]:
# Inference for a specific image
results = model.predict(source="/content/datasets/fashion-mnist/test/9/1007.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()

        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/9/1007.png: 32x32 9 0.95, 7 0.05, 5 0.00, 8 0.00, 3 0.00, 6.5ms
Speed: 13.8ms preprocess, 6.5ms inference, 0.2ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train2
Predicted class: Ankle Boot, Confidence: 0.95


In [ ]:
# Inference for a more images
results = model.predict(source="/content/datasets/fashion-mnist/test/0/1049.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/0/1049.png: 32x32 0 0.94, 6 0.06, 3 0.00, 2 0.00, 4 0.00, 3.7ms
Speed: 1.8ms preprocess, 3.7ms inference, 0.1ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train3
Predicted class: T-shirt/top, Confidence: 0.94


In [ ]:
# Inference for a more images
results = model.predict(source="/content/datasets/fashion-mnist/test/7/1040.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/7/1040.png: 32x32 7 1.00, 5 0.00, 9 0.00, 8 0.00, 0 0.00, 6.2ms
Speed: 2.5ms preprocess, 6.2ms inference, 0.1ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train4
Predicted class: Sneaker, Confidence: 1.00


In [ ]:
# Inference for a more images
results = model.predict(source="/content/datasets/fashion-mnist/test/8/1564.png", save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        # Get the index of the top predicted class
        predicted_class_idx = result.probs.top1
        # Map the class index to the class name using the previously defined list
        predicted_class_name = class_names[predicted_class_idx]
        # Get the confidence score for the top class
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



image 1/1 /content/datasets/fashion-mnist/test/8/1564.png: 32x32 8 1.00, 4 0.00, 0 0.00, 6 0.00, 2 0.00, 7.5ms
Speed: 2.2ms preprocess, 7.5ms inference, 0.1ms postprocess per image at shape (1, 3, 32, 32)
Results saved to runs/classify/train5
Predicted class: Bag, Confidence: 1.00


### Testing on Google Images

In [ ]:
# Using an image from the web
image_url = "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?q=80&w=1780&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Download the image from the URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Preprocessing the image for YOLO
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1))
])
img_tensor = transform(img).unsqueeze(0)

# Perform inference
results = model.predict(source=img_tensor, save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        predicted_class_idx = result.probs.top1
        predicted_class_name = class_names[predicted_class_idx]
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")



0: 640x640 6 0.32, 8 0.16, 2 0.11, 0 0.11, 4 0.11, 134.8ms
Speed: 0.0ms preprocess, 134.8ms inference, 12.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/train6
Predicted class: Shirt, Confidence: 0.32


# **STYLING ADVICE PORTION**
###**PARSING THE IMAGE TO AN LLM TO GIVE STYLING ADVICE**
The LLM chosen is OpenAI 's ChatGPT

###Testing my API and GPT 4's advice to see if the advice is suitable for the user

In [ ]:
# Set your API key
client =OpenAI(api_key="INSERT-API-KEY")

response = client.chat.completions.create(
  model="gpt-4",
  messages=[
      {'role': 'user',
       'content': 'You are a fashion expert. Suggest stylish outfit combinations for someone wearing a shirt.'}
  ]
)

message = response.choices[0].message.content
print(message)


1. Denim Chic: Team up the shirt with a pair of blue skinny jeans and white low-top sneakers for a casual yet stylish look. Don't forget to accessorize with a minimalistic pendant and watch.

2. Sophisticated Business: Wear your shirt with a well-tailored suit, a pair of oxfords, and a nice tie. This outfit is perfect for a meeting or business affair.

3. Summer Breeze: Pair a light colored shirt with cotton shorts, adding espadrilles and sunglasses for a perfect summer day look.

4. Cozy Layering: Wear the shirt under a crew-neck sweater, pair it with chinos and monk strap shoes for a relaxed yet refined appearance.

5. Casual Friday: For a trendy office look, pair up your shirt with black jeans, add a leather belt, and loafers. Roll up the shirt sleeves for a cool, casual flair.

6. Trendy Hipster: Wear your shirt with distressed skinny jeans, and add some stylishly rugged boots. Layer with a denim jacket to amp up the hipster vibe.

7. Classic and Dapper: Button up your shirt and pa

### Trying to test the code to give fashion advice on a google image after the url has been inputed.

In [ ]:
# Using an image from the web
image_url = "https://images.unsplash.com/photo-1521572163474-6864f9cf17ab?q=80&w=1780&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D"

# Download the image from the URL
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# Preprocessing the image for YOLO
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1))
])
img_tensor = transform(img).unsqueeze(0)

# Perform inference
results = model.predict(source=img_tensor, save=True)

# Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        predicted_class_idx = result.probs.top1
        predicted_class_name = class_names[predicted_class_idx]
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")

# Set your API key
client =OpenAI(api_key="INSERT-API-KEY")

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {'role': 'user',
         'content': f'You are a fashion expert. Suggest stylish outfit combinations for someone wearing a {predicted_class_name}.'}
    ]
)

message = response.choices[0].message.content
print(message)




0: 640x640 6 0.32, 8 0.16, 2 0.11, 0 0.11, 4 0.11, 114.9ms
Speed: 0.0ms preprocess, 114.9ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/train7
Predicted class: Shirt, Confidence: 0.32
1. Shirt + Straight-leg Jeans + White Sneakers: This combo is perfect for a casual day-out. Add a leather watch or bracelet to take your outfit to the next level.

2. Shirt + Blazer + Dress Pants + Dress Shoes: Ideal for a business-casual or professional setting. Opt for solid color shirts and keep your blazer and pants in matching shades. A classic pair of dress shoes will seal this elegant look.

3. Shirt + Jumper + Chinos + Loafers: A combination of a shirt under a jumper (wool or cotton), paired with chinos and loafers offers a cozy yet smart look. This outfit is great for autumn.

4. Shirt + Leather Jacket + Black Jeans + Boots: For a bit of an edge, wearing a shirt under a leather jacket with black jeans and boots is a great pick. You'll look effo

### Giving fashion advice after the user uploads their own image
Will be uploading the image of a bag i found online

In [ ]:
# Step 1: Upload the image
uploaded = files.upload()

Saving a bag.jpg to a bag (1).jpg


In [ ]:
# Get the uploaded file path
image_path = list(uploaded.keys())[0]

# Open the uploaded image
img = Image.open(image_path)

# Step 2: Preprocessing the image for YOLO
transform = transforms.Compose([
    transforms.Resize((640, 640)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x if x.shape[0] == 3 else x.repeat(3, 1, 1))
])

img_tensor = transform(img).unsqueeze(0)

# Step 3: YOLO Prediction (Assuming 'model' is defined already)
results = model.predict(source=img_tensor, save=True)

# Step 4: Process the results
for result in results:
    if hasattr(result, "probs") and result.probs is not None:
        predicted_class_idx = result.probs.top1
        predicted_class_name = class_names[predicted_class_idx]
        confidence = result.probs.top1conf.item()
        print(f"Predicted class: {predicted_class_name}, Confidence: {confidence:.2f}")
    else:
        print("No predictions for this image.")

# Step 5: Ask OpenAI for fashion advice based on the predicted class
client =OpenAI(api_key="INSERT-API-KEY")


response = client.chat.completions.create(
    model="gpt-4",
    messages=[{
        'role': 'user',
        'content': f'You are a fashion expert. Suggest stylish outfit combinations for someone wearing a {predicted_class_name}.'}]
)

message = response.choices[0].message.content
print(message)


0: 640x640 6 0.36, 4 0.16, 8 0.15, 2 0.10, 3 0.09, 111.3ms
Speed: 0.1ms preprocess, 111.3ms inference, 6.4ms postprocess per image at shape (1, 3, 640, 640)
Results saved to runs/classify/train12
Predicted class: Shirt, Confidence: 0.36
1. Casual Look: A shirt in checked or solid colors paired with dark denim jeans. Suede loafers or sneakers would finish off the look perfectly. 

2. Formal Look: A crisp white shirt paired with slim-fit black trousers. These can be accessorized with a monochromatic tie, leather belt, and a pair of patent leather oxfords. 

3. Semi-Formal Look: A pastel-colored shirt paired with chinos. Match it with a leather belt and a pair of loafers or brogues. 

4. Date Look: A dark-colored shirt (preferably black or navy) paired with a tailored blazer and slim-fit jeans. Ankle boots or loafers would complete this outfit.

5. Summer Look: A bright, short-sleeved shirt with white or light-colored linen trousers. Complete the look with a pair of white sneakers.

6. W